## NLP Project



In [2]:
import json

from pathlib import Path


def create_paper_dict(paper):
    paper_dict = {}
    abstract = ''
    text = ''
    
    try:  # many papers don't have abstracts
        for i in paper['abstract']:
            abstract += i['text']
    except:
        pass
    for i in paper['body_text']:
        text += i['text']
    
    paper_dict['paper_id'] = paper['paper_id']
    paper_dict['abstract'] = abstract
    paper_dict['body_text'] = text
    
    return paper_dict

data_path = 'C://Users//Binyamin//PythonProjects//NLP//final_project//data//'
lit = []
pathlist = Path(data_path).glob('**/*.json')
for path in pathlist:
    path_in_str = str(path)  # because path is object not string
    with open(path_in_str) as f:
        data = json.load(f)
    paper_dict = create_paper_dict(data)
    lit.append(paper_dict)

In [3]:
len(lit)

71261

In [28]:
import time


In [36]:
ls ..\data

 Volume in drive C is Acer
 Volume Serial Number is BA05-5E0F

 Directory of C:\Users\Binyamin\PythonProjects\NLP\final_project\data

05/10/2020  02:11 PM    <DIR>          .
05/10/2020  02:11 PM    <DIR>          ..
05/08/2020  03:55 PM    <DIR>          arxiv
05/08/2020  03:55 PM    <DIR>          biorxiv_medrxiv
05/10/2020  12:40 PM    <DIR>          comm_use_subset
05/08/2020  04:39 PM    <DIR>          cord_19_embeddings_4_24
05/02/2020  08:39 PM            26,690 COVID.DATA.LIC.AGMT.pdf
05/08/2020  03:58 PM    <DIR>          custom_license
05/02/2020  08:49 PM             2,906 json_schema.txt
05/02/2020  08:49 PM        89,290,114 metadata.csv
05/02/2020  08:50 PM             5,434 metadata.readme
05/08/2020  04:02 PM    <DIR>          noncomm_use_subset
05/10/2020  02:14 PM     2,076,181,339 paper_dicts
               5 File(s)  2,165,506,483 bytes
               8 Dir(s)  21,870,022,656 bytes free


In [63]:
f = open('../data/single_text_file.txt', 'a', encoding='UTF-8')
all_text = ''
t_time = 0
for num, i in enumerate(lit):
    start = time.perf_counter()
    all_text = all_text+i['abstract']
    all_text = all_text+i['body_text']
    end = time.perf_counter()
    t_time = t_time + (end-start)
    if num%500 == 0:
        f.write(all_text)
        all_text = ''
        print ("done ", num, " papers in ", t_time, " seconds")
        t_time = 0

done  0  papers in  0.0067048000055365264  seconds
done  500  papers in  10.631782699696487  seconds
done  1000  papers in  7.8358053001138614  seconds
done  1500  papers in  7.038967199987383  seconds
done  2000  papers in  6.566383399927872  seconds
done  2500  papers in  6.56111050004256  seconds
done  3000  papers in  6.073433299810858  seconds
done  3500  papers in  6.77269339976192  seconds
done  4000  papers in  9.315488700071  seconds
done  4500  papers in  9.450686000156566  seconds
done  5000  papers in  10.072518799919635  seconds
done  5500  papers in  9.106466499943053  seconds
done  6000  papers in  8.344403199997032  seconds
done  6500  papers in  8.469056899935822  seconds
done  7000  papers in  8.367996099987067  seconds
done  7500  papers in  8.089393200047198  seconds
done  8000  papers in  8.690419099802966  seconds
done  8500  papers in  8.256924200148205  seconds
done  9000  papers in  8.280585600063205  seconds
done  9500  papers in  8.39931279992743  seconds
don

In [23]:
done  0  papers in  1.800013706088066e-06  seconds
done  1000  papers in  23.865991999962716  seconds
done  2000  papers in  68.89297090018226  seconds
done  3000  papers in  110.14684130004025  seconds
done  4000  papers in  166.1707331002399  seconds

SyntaxError: invalid syntax (<ipython-input-23-28aca39aaa8a>, line 1)

In [64]:
f.close()

In [42]:
f = open('../data/single_text_file.txt', 'a', encoding='UTF-8')
f.write(all_text)
all_text = ''
f.close()

In [69]:
f = open('../data/single_text_file.txt', 'r', encoding='UTF-8')

In [ ]:
t = f.read()

In [68]:
len(t)/len(lit)

0.0

NameError: name 'size' is not defined